### Embeddings on summary nodes

In [1]:
from src.indexing_utils import load_docs
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

Settings.embed_model = GeminiEmbedding(model_name='models/embedding-001')
Settings.llm = Gemini(model_name='models/gemini-pro', temperature=0.5)

In [7]:
nodes = load_docs(path="../../../storage/docstore/summary_node_with_embeddings", return_docstore=False)
doc_store = load_docs(path="../../../storage/docstore/summary_node_with_embeddings", return_docstore=True)

In [ ]:
# from llama_index.core import StorageContext

# # create storage context using default stores
# storage_context = StorageContext.from_defaults(docstore=nodes)

In [3]:
index = VectorStoreIndex(nodes=nodes, embed_model=Settings.embed_model, show_progress=True)

# store it for later
# index.storage_context.persist(persist_dir='../../../storage/vectorstore/summa')

Generating embeddings: 0it [00:00, ?it/s]

In [4]:
query_engine = index.as_query_engine()

In [12]:
response = query_engine.query("How to deal with muscle and body pain in Ayurveda?")

In [13]:
from llama_index.legacy.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)

Final Response: The provided context does not contain any information
on how to deal with muscle and body pain in Ayurveda.
______________________________________________________________________
Source Node 1/2
Node ID: f0153eec-5bad-435c-be45-fcd89561dc29
Similarity: 0.8268947330344362
Text: Reference: M Ni. Paris`hisht`a Nirukti: Sandhaaran`aat
S`hareerasya Taa Aapah Soma Samdnyitaah | Tatah Somakshayaat
Streen`aam Somaroga Iti Smritah || M Ni. Paris`hisht`a As body is
sustained due to body fluids, it is called as Soma. This Soma in
females gets depleted through the urinary tract; therefore this
disease is called as Somaroga. Hetu:...
______________________________________________________________________
Source Node 2/2
Node ID: ff3570a0-de39-44e0-84cd-54f3cf828366
Similarity: 0.8038234877479593
Text: Somaroga Chikitsaa Reference: B. R. 87, Y.R. Somaroga Chikitsaa
(Uttaraadha) Introduction: Chikitsaa Similar To: Pathya: Apathya:
Search Anveshaka: Search Dravya, Kalpa, Aahaara, Vihaar

In [11]:
doc_store.get_node('f0153eec-5bad-435c-be45-fcd89561dc29').metadata['summary']

'Ayurvedic medicine, a traditional Indian system of medicine, emphasizes the balance of three doshas: Vata, Pitta, and Kapha. These doshas are believed to govern various functions in the body, and imbalances can lead to disease. Ayurveda also places great importance on diet and lifestyle, with specific recommendations for different body types and conditions.'

In [15]:
doc_store.get_node('ff3570a0-de39-44e0-84cd-54f3cf828366').text

'Somaroga Chikitsaa Reference: B. R. 87, Y.R. Somaroga Chikitsaa (Uttaraadha) Introduction: Chikitsaa Similar To: Pathya: Apathya: Search Anveshaka: Search Dravya, Kalpa, Aahaara, Vihaara, Chikitsaa Karma For'

In [16]:
doc_store.delete_document('ff3570a0-de39-44e0-84cd-54f3cf828366')